# Ising Model

## construct a 3-qubit state

#### Construct basic qubit by Statevector

U can contruct by 2 following forms:

- 1 psi = one.tensor(zero.tensor(zero))
  - which means "$1\otimes0\otimes0$"
- 2 psi = one ^ zero ^ zero
  - also means "$1\otimes0\otimes0$"

In [3]:
from qiskit.quantum_info import Statevector
import numpy as np
import pylatexenc

zero = Statevector([1,0])
print("zero	is", zero)
one = Statevector([0,1])

psi = one.tensor(zero.tensor(zero))
print('psi is', psi)

psi = one^zero^zero
psi.draw("latex")

psi = 1/np.sqrt(2) * (zero^zero^zero) + 1/np.sqrt(2) * (one^one^one)
psi.draw("latex")

zero	is Statevector([1.+0.j, 0.+0.j],
            dims=(2,))
psi is Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


<IPython.core.display.Latex object>

## Construct a Pauli state

This step is acutually construct *Cost Hamiltonian* for encoding

In [4]:
from qiskit.quantum_info import Pauli

In [5]:
z0z1 = Pauli("ZZI")
print("z0z1 is", z0z1)
print('and its matrix is')
print(z0z1.to_matrix())

z0z1 is ZZI
and its matrix is
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j]]


In [6]:
print('The sparse representation of z0z1 is')
print(z0z1.to_matrix(sparse=True))

The sparse representation of z0z1 is
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 8 stored elements and shape (8, 8)>
  Coords	Values
  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(-1+0j)
  (3, 3)	(-1+0j)
  (4, 4)	(-1+0j)
  (5, 5)	(-1+0j)
  (6, 6)	(1+0j)
  (7, 7)	(1+0j)


## Operate a Pauli state like Z or I

In [7]:
from qiskit.quantum_info import SparsePauliOp as PauliOp

In [8]:
H_cut = PauliOp(Pauli("ZZI")) + PauliOp(Pauli("ZIZ"))
print('H_cut is')
print(H_cut)
print('The sparse representation of H_cut is')
print(H_cut.to_matrix(sparse=True))

H_cut is
SparsePauliOp(['ZZI', 'ZIZ'],
              coeffs=[1.+0.j, 1.+0.j])
The sparse representation of H_cut is
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 8 stored elements and shape (8, 8)>
  Coords	Values
  (0, 0)	(2+0j)
  (1, 1)	0j
  (2, 2)	0j
  (3, 3)	(-2+0j)
  (4, 4)	(-2+0j)
  (5, 5)	0j
  (6, 6)	0j
  (7, 7)	(2+0j)


## Construct a linear combination of Hamiltonian

In [2]:
from qiskit import QuantumCircuit

In [1]:
# 所有三比特的基态
basis_states = ['000', '001', '010', '011', '100', '101', '110', '111']

In [9]:
for state_str in basis_states:
    # 创建对应的量子电路
    qc = QuantumCircuit(3)
    # 初始化到对应的基态
    qc.initialize([1,0], 0)
    qc.initialize([0,1], 1)
    qc.initialize([1,0], 2)
    # 或者直接用Statevector
    state = Statevector.from_label(state_str)
    E = state.expectation_value(H_cut)
    print(f"State |{state_str}>: Expectation value = {E}")


State |000>: Expectation value = (2+0j)
State |001>: Expectation value = 0j
State |010>: Expectation value = 0j
State |011>: Expectation value = (-2+0j)
State |100>: Expectation value = (-2+0j)
State |101>: Expectation value = 0j
State |110>: Expectation value = 0j
State |111>: Expectation value = (2+0j)


In [10]:
psi = one^zero^zero
E = psi.expectation_value(H_cut)
print(E)

(-2+0j)
